In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image #cv2.puttext용함수. 실제로는 지울 것

#한국어 인식함수 (실제 사용시 삭제)
def myPutText(src,text,pos,font_size,font_color):
    img_pil=Image.fromarray(src)
    draw=ImageDraw.Draw(img_pil)
    font=ImageFont.truetype('한국기계연구원_bold.ttf',font_size)
    draw.text(pos,text,font=font,fill=font_color)
    return np.array(img_pil)

max_num_hands = 1

# gesture = {
#     0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f',
#     6:'g', 7:'h', 8:'i', 9:'j', 10:'k',11:'l', 12:'m', 13:'n', 14:'o', 15:'p', 16:'q',
#     17:'r', 18:'s', 19:'t', 20:'u', 21:'v', 22:'w', 23:'x', 24:'y', 25:'z', 26:'aa' , 27:'bb'
# }


gesture = {
    0:'ㄱ', 1:'ㄴ', 2:'ㄷ' , 3:'ㄹ' , 4:'ㅁ' , 5:'ㅂ', 6:'ㅅ', 7:'ㅇ', 8:'ㅈ', 9:'ㅊ', 10:'ㅋ', 11:'ㅌ', 12:'ㅍ',
   13:'ㅎ', 14:'ㅏ', 15:'ㅑ', 16:'ㅓ', 17:'ㅕ', 18:'ㅗ', 19:'ㅛ', 20:'ㅜ', 21:'ㅠ', 22:'ㅡ', 23:'ㅣ', 24:'ㅐ', 25:'ㅒ', 26:'ㅔ', 27:'ㅖ',
   28:'ㅚ', 29:'ㅟ', 30:'ㅢ'
}




# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Gesture recognition model
file = np.genfromtxt('dataset/dataSet_v1.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)

#모델을 만들고 바로 사용하는 코드가 섞여있다.
# 1. 데이터셋 만드는 방법 찾기 (O)
# 2. 데이터셋 추가하고 라벨도 추가해서 실제 인식이 되는지 확인하기 (커스텀 가능성 확인) (O)
# 3. 데이터셋을 하나의 파일이 아니라 분리하는 방법도?? 찾아서 모듈화 하기 (각 라벨마다 어떤 정보를 넣는지 통일할 필요가 있음.) (O)



# 4. 입력값을 영상이 아니라 정적인 21개의 데이터 배열로 테스트 해보기
# 5. 배열 50개를 가지고 가장 높은 정답라벨을 출력하도록 코드 수정하기
# 6. 동시에 응답이 잘 나오도록 영상에서 데이터 뽑는 코드는 그대로 유지할 수 있어야 함

# 7. 모델을 분리하고 모델을 생성할때 정확도 정보를 뽑아낼 수 있는지 찾아보기 (이전 모델 학습 코드를 재활용 해도 됨)
# 8. 분리된 모델 테스트
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        continue

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        rps_result = []

        for res in result.multi_hand_landmarks:
#             print(res.landmark)

            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            # Inference gesture
            data = np.array([angle], dtype=np.float32)
            # print("Data :",data)
            ret, results, neighbours, dist = knn.findNearest(data, 3)
            idx = int(results[0][0])

            # Draw gesture result
            if idx in gesture.keys():
                org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0]))
                

                rps_result.append({
                    'rps': gesture[idx],
                    'org': org
                })
               
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            img=myPutText(img,gesture[idx],(int(img.shape[1]/2), 30),100,(0,255,0)) #확인용 코드 , 실사용시 삭제필요 
      
    cv2.imshow('temp', img)
    if cv2.waitKey(1) == ord('q'):
        break